In [127]:
import os
import datetime
from jinja2 import Template, Environment, FileSystemLoader
from PIL import Image
  


In [128]:
file_path = "chat.txt"
with open(file_path, mode='r', encoding="utf8") as f:
    data = f.readlines()

In [129]:
utente = "Otti" #input("Inserisci il received: ")

In [130]:
dataset = data [1:]
cleaned_data = []
lista = []

for line in dataset:
    # Check, whether it is a new line or not
    # If the following characters are in the line -> assumption it is NOT a new line
    if '/' in line and ':' in line and ',' in line and '[' in line:
        # grab the info and cut it out
        date = line.split("[")[1].split(",")[0]
        line2 = line[len(date):]
        time = line2.split(",")[1].split("]")[0]
        line3 = line2[len(time):]
        sender = line3.split("]")[1].split(":")[0]
        line4 = line3[len(sender):]
        message = line4[5:-1]
        message = message.replace(" \u200e", "" ).replace("\n", "")
        position = 'received'
        if sender[1:] == utente:
            position = 'sent'
        cleaned_data.append([position, date, time[1:], sender[1:], message])
        lista.append(date)
    # else, assumption -> new line. Append new line to previous 'message'
    else:
        new = cleaned_data[-1][-1] + " " + line
        cleaned_data[-1][-1] = new

In [131]:
result = [] 
[result.append(x) for x in lista if x not in result]

[None, None, None]

In [132]:
f.close()

In [133]:
file_html_path = "index_wa.html"
if os.path.exists(file_html_path):
    os.remove(file_html_path) 
i = open(file_html_path, mode='x', encoding="utf8")
#i.write('<!DOCTYPE html><html><head></head><body>')
#i.write('test '+ str(datetime.date.today())+"\n")
#i.write('<table border="0px solid black" style="width:100%">')
#i.write('<th>1</th><th>2</th><th>3</th><th>4</th><th>5</th><th>6</th><th>7</th><th>8</th>')


#for r in cleaned_data:
#    
#    if r[3] != utente:
#        i.write('<tr><td>'+r[1]+" "+r[2]+" "+r[3]+'</td></tr>')
#        i.write('<tr><td colspan=4>'+r[4]+'</td></tr>')
    
#    else:
#        i.write('<tr><td colspan=7></td><td align="right">'+r[1]+" "+r[2]+" "+r[3]+'<td></tr>')
#        i.write('<tr><td colspan=4></td>')
#        i.write('<td colspan=4 align="right">'+r[4]+'</td></tr>')

#i.write('</table>')
#i.write('</body></html>')


In [134]:
file_loader = FileSystemLoader("templates")
env = Environment(loader=file_loader)
inizio=env.get_template("inizio.txt")
fine=env.get_template("fine.txt")
data_template=env.get_template("data_template.txt")
message_template=env.get_template("message_template.txt")
media_template=env.get_template("media_template.txt")

In [135]:
i.write(inizio.render())

1327

In [136]:
data_messaggio = ""

for m in cleaned_data:
    mess = m[4]
    
    if m[1] != data_messaggio:
        data_messaggio = m[1]
        i.write((data_template.render(data=m[1])))
    
    #posizione conterrà l'inizio della stringa "<allegato:"
    posizione = m[4].find("<allegato:")
    
    if posizione>-1:
        if(m[4].find(".jpg")>-1):
            filename = m[4][posizione+11:len(m[4])-1]
            with Image.open(filename) as image:
                width, height = image.size
            print(str(width) + " " + str(height))
   
            mess =  "<a href=" + m[4][posizione+11:len(m[4])-1]+ " data-lightbox=" + m[4][posizione+11:len(m[4])-1]+ "  >" + "<img src=\""+m[4][posizione+11:len(m[4])-1]+"\">" +"</a>"
   
        elif(m[4].find(".opus") >-1 or m[4].find(".mp3") >-1):
            mess = "<audio controls><source src="+ m[4][posizione+11:len(m[4])-1] +  " type='audio/ogg'>Your browser does not support the audio element.</audio>"
        else:
            mess = "<a href=\""+m[4][posizione+11:len(m[4])-1]+"\">"+m[4][posizione+11:len(m[4])-1]+"</a>"
        i.write(media_template.render(tipo=m[0], messaggio=mess,ora=m[2][0:5]))
        
    else:
        i.write(message_template.render(tipo=m[0], messaggio=mess,ora=m[2][0:5]))

1578 2048
225 224


In [137]:
i.write(fine.render())
i.close()